In [238]:
import codecs
import re
import numpy as np
import pydot_ng as pydot

In [3]:
with codecs.open("train/text/neko.txt.cabocha", "r", encoding="utf-8") as f:
    result = f.readlines()
result

['* 0 -1D 0/0 0.000000\n',
 '一\t名詞,数,*,*,*,*,一,イチ,イチ\n',
 'EOS\n',
 'EOS\n',
 '* 0 2D 0/0 -0.764522\n',
 '\u3000\t記号,空白,*,*,*,*,\u3000,\u3000,\u3000\n',
 '* 1 2D 0/1 -0.764522\n',
 '吾輩\t名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ\n',
 'は\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n',
 '* 2 -1D 0/2 0.000000\n',
 '猫\t名詞,一般,*,*,*,*,猫,ネコ,ネコ\n',
 'で\t助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ\n',
 'ある\t助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル\n',
 '。\t記号,句点,*,*,*,*,。,。,。\n',
 'EOS\n',
 '* 0 2D 0/1 -1.911675\n',
 '名前\t名詞,一般,*,*,*,*,名前,ナマエ,ナマエ\n',
 'は\t助詞,係助詞,*,*,*,*,は,ハ,ワ\n',
 '* 1 2D 0/0 -1.911675\n',
 'まだ\t副詞,助詞類接続,*,*,*,*,まだ,マダ,マダ\n',
 '* 2 -1D 0/0 0.000000\n',
 '無い\t形容詞,自立,*,*,形容詞・アウオ段,基本形,無い,ナイ,ナイ\n',
 '。\t記号,句点,*,*,*,*,。,。,。\n',
 'EOS\n',
 'EOS\n',
 '* 0 1D 1/2 1.058678\n',
 '\u3000\t記号,空白,*,*,*,*,\u3000,\u3000,\u3000\n',
 'どこ\t名詞,代名詞,一般,*,*,*,どこ,ドコ,ドコ\n',
 'で\t助詞,格助詞,一般,*,*,*,で,デ,デ\n',
 '* 1 4D 0/2 -1.453749\n',
 '生れ\t動詞,自立,*,*,一段,連用形,生れる,ウマレ,ウマレ\n',
 'た\t助動詞,*,*,*,特殊・タ,基本形,た,タ,タ\n',
 'か\t助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ\n',
 '* 2 4D 0/0 -1.453749\n'

In [9]:
class Morph():
    
    def __init__(self, result):
        self.surface = result.split()[0]
        self.base = result.split()[1].split(",")[6]
        self.pos = result.split()[1].split(",")[0]
        self.pos1 = result.split()[1].split(",")[1]

In [16]:
def nlp40():
    Morph_ls = []
    sentence_ls = []
    with codecs.open("train/text/neko.txt.cabocha", "r", encoding="utf-8") as f:
        results = f.readlines()
    for result in results:
        if result.startswith("EOS"):
            if len(Morph_ls) < 1:
                continue
            else:
                sentence_ls.append(Morph_ls)
                Morph_ls = []
                continue
        if result.startswith("*"):
            continue
        if re.match(r"\s", result):
            continue
        morph = Morph(result)
        Morph_ls.append(morph)
    return sentence_ls


if __name__ == "__main__":
    print([i.surface for i in nlp40()[3]])

['どこ', 'で', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '。']


In [211]:
class Chunk():
    def __init__(self, result):
        self.morphs = []
        self.dst = int(re.findall(r"-?[0-9]+", result.split()[2])[0])
        self.srcs = []
        self.ind = int(result.split()[1])

In [310]:
def nlp41():
    Chunk_ls = []
    dst_dict = {}
    Morph_ls = []
    sentence_ls = []
    space_ls = []
    with codecs.open("train/text/neko.txt.cabocha", "r", encoding="utf-8") as f:
        results = f.readlines()
    chunk_ind = 0
    for result in results:
        if result.startswith("EOS"):
            if len(Chunk_ls) < 1:
                Chunk_ls = []
                Morph_ls = []
                continue
            else:
                chunk.morphs = Morph_ls
                dst_dict[chunk_ind] = chunk.dst
                Chunk_ls.append(chunk)
                for i, _Chunk in enumerate(Chunk_ls):
                    if _Chunk == "":
                        continue
                    dists = np.asarray(list(dst_dict.values()))
                    srcs = np.asarray(list(dst_dict.keys()))
                    _Chunk.srcs = list(srcs[dists == i])
                Chunk_ls = [
                    _chunk for i, _chunk in enumerate(Chunk_ls)
                    if _chunk != ""]
                sentence_ls.append(Chunk_ls)
                Chunk_ls = []
                space_ls = []
                Morph_ls = []
                dst_dict = {}
                continue
        if result.startswith("*"):
            if len(Morph_ls) > 0:
                chunk.morphs = Morph_ls
                Chunk_ls.append(chunk)
                dst_dict[chunk_ind] = chunk.dst
            chunk_ind = int(result.split()[1])
            chunk = Chunk(result)
            Morph_ls = []
            continue
        if re.match(r"\s", result) is not None:
            Morph_ls.append("")
            continue
        Morph_ls.append(Morph(result))
    return sentence_ls


if __name__ == "__main__":
    result = nlp41()[1]
    for i in result:
        for j in i.morphs:
            if j != "":
                print(j.surface + " " , i.dst, i.srcs)

名前  2 []
は  2 []
まだ  2 []
無い  -1 [0, 1]
。  -1 [0, 1]


In [225]:
def nlp42():
    sentence_ls = nlp41()
    src_dst_ls = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        surfaces = np.asarray([
            morphs_face(_Chunk.morphs) for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if _Chunk.dst == -1:
                dst = ""
            else:
                dst = "".join(surfaces[inds == _Chunk.dst].tolist())
            src = morphs_face(_Chunk.morphs)
            src_dst_ls.append(src + "\t" + dst)
    return src_dst_ls


if __name__ == "__main__":
    print(nlp42()[:5])

['吾輩は\t猫である。', '猫である。\t', '名前は\t無い。', 'まだ\t無い。', '無い。\t']


In [665]:
def morphs_face(morphs, replace=False):
    face = ""
    for morph in morphs:
        if morph == "":
            continue
        if re.match(r"\w", morph.surface) is None:
            continue
        if replace:
            face += morph.surface if morph.pos != "名詞" else "X"
        else:
            face += morph.surface
    return face

def morphs_poses(morphs):
    poses = []
    for morph in morphs:
        if morph == "":
            continue
        if re.match(r"\w", morph.surface) is None:
            continue
        poses.append(morph.pos)
    return poses

def morphs_pos1s(morphs):
    pos1s = []
    for morph in morphs:
        if morph == "":
            continue
        if re.match(r"\w", morph.surface) is None:
            continue
        pos1s.append(morph.pos1)
    return pos1s

def morphs_base_ls(morphs):
    base_ls = []
    for morph in morphs:
        if morph == "":
            continue
        if re.match(r"\w", morph.surface) is None:
            continue
        base_ls.append(morph.base)
    return base_ls

def morphs_face_ls(morphs):
    face_ls = []
    for morph in morphs:
        if morph == "":
            continue
        if re.match(r"\w", morph.surface) is None:
            continue
        face_ls.append(morph.surface)
    return face_ls

In [231]:
def nlp43():
    sentence_ls = nlp41()
    dst_verb_ls = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        surfaces = np.asarray([
            morphs_face(_Chunk.morphs) for _Chunk in Chunk_ls])
        poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if _Chunk.dst == -1:
                continue
            else:
                src_poses = sum(
                    poses[inds == _Chunk.ind].tolist(),[])
                dst_poses = sum(
                    poses[inds == _Chunk.dst].tolist(),[])
                if "動詞" not in dst_poses:
                    continue
                if "名詞" not in src_poses:
                    continue
                dst = "".join(
                    surfaces[inds == _Chunk.dst].tolist())
            src = morphs_face(_Chunk.morphs)
            dst_verb_ls.append(src + "\t" + dst)
    return dst_verb_ls


if __name__ == "__main__":
    print(nlp43()[:5])

['どこで\t生れたか', '見当が\tつかぬ', '所で\t泣いて', 'ニャーニャー\t泣いて', 'いた事だけは\t記憶している']


In [237]:
graph = pydot.Dot(graph_type='digraph')
id1 = 1
id2 = 2
label1 = "1"
label2 = "2"
graph.add_node(pydot.Node(id1, label=label1))
graph.add_node(pydot.Node(id2, label=label2))
graph.add_edge(pydot.Edge(id1, id2))
graph.write_png("result.png")

True

In [262]:
def nlp44():
    sentence_ls = nlp41()
    src_dst_ls = []
    src_dst_ls_sentence = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        chunks = np.asarray([_Chunk for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if _Chunk.dst == -1:
                dst = ""
            else:
                dst = chunks[inds == _Chunk.dst][0]
            src_dst_ls.append([_Chunk,dst])
        src_dst_ls_sentence.append(src_dst_ls)
        src_dst_ls = []
    return src_dst_ls_sentence

def graph_from_src_dst(src_dst_ls_sentence):
    for i, src_dst_ls in enumerate(src_dst_ls_sentence):
        node_ls = []
        graph = pydot.Dot(graph_type='digraph')
        for src_dst in src_dst_ls:
            src = src_dst[0]
            dst = src_dst[1]
            if src.ind not in node_ls:
                node_ls.append(src.ind)
                graph.add_node(pydot.Node(
                    src.ind, label=morphs_face(src.morphs)))
            if dst == "":
                continue
            if dst.ind not in node_ls:
                node_ls.append(dst.ind)
                graph.add_node(pydot.Node(
                    dst.ind, label=morphs_face(dst.morphs)))
            graph.add_edge(pydot.Edge(src.ind, dst.ind))
        graph.write_png("train/pic/result%i.png"%(i))
        
        
if __name__ == "__main__":
    src_dst_ls_sentence = nlp44()
    graph_from_src_dst(src_dst_ls_sentence)

In [316]:
def nlp45():
    sentence_ls = nlp41()
    kaku_ls = []
    verb_kaku = ""
    src_joshi = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        bases = np.asarray([
            morphs_base_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        surfaces = np.asarray([
            morphs_face_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if len(_Chunk.srcs) == 0:
                continue
            else:
                src_joshi = []
                verb_kaku = ""
                dst_poses = sum(
                    poses[inds == _Chunk.ind].tolist(),[])
                if "動詞" not in dst_poses:
                    continue
                dst_base = sum(
                        bases[inds == _Chunk.ind].tolist(), [])
                verb_kaku += dst_base[dst_poses.index("動詞")] + \
                                "\t"
                for src in _Chunk.srcs:
                    src_pos = sum(
                        poses[inds == src].tolist(),[])
                    if "助詞" not in src_pos:
                        continue
                    src_bases = sum(
                        bases[inds == src].tolist(), [])
                    src_surfaces = sum(
                        surfaces[inds == src].tolist(), [])
                    src_joshi.append(src_bases[src_pos.index("助詞")])
                verb_kaku += " ".join(src_joshi)
                kaku_ls.append(verb_kaku)
    return kaku_ls


if __name__ == "__main__":
    print(nlp45()[:10])

['生れる\tで', 'つく\tか が', '泣く\tで', 'する\tて だけ', '始める\tで', '見る\tは を', '聞く\tで', '捕える\tを', '煮る\tて', '食う\tて']


In [318]:
def nlp46():
    sentence_ls = nlp41()
    kaku_ls = []
    verb_kaku = ""
    src_joshi = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        bases = np.asarray([
            morphs_base_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        surfaces = np.asarray([
            morphs_face_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if len(_Chunk.srcs) == 0:
                continue
            else:
                src_kou = []
                src_joshi = []
                verb_kaku = ""
                dst_poses = sum(
                    poses[inds == _Chunk.ind].tolist(),[])
                if "動詞" not in dst_poses:
                    continue
                dst_base = sum(
                        bases[inds == _Chunk.ind].tolist(), [])
                verb_kaku += dst_base[dst_poses.index("動詞")] + \
                                "\t"
                for src in _Chunk.srcs:
                    src_pos = sum(
                        poses[inds == src].tolist(),[])
                    if "助詞" not in src_pos:
                        continue
                    src_bases = sum(
                        bases[inds == src].tolist(), [])
                    src_surfaces = sum(
                        surfaces[inds == src].tolist(), [])
                    src_joshi.append(src_surfaces[src_pos.index("助詞")])
                    src_kou.append("".join(src_surfaces))
                verb_kaku += " ".join(src_joshi)
                verb_kaku += "\t" + " ".join(src_kou)
                kaku_ls.append(verb_kaku)
    return kaku_ls


if __name__ == "__main__":
    print(nlp46()[:10])

['生れる\tで\tどこで', 'つく\tか が\t生れたか 見当が', '泣く\tで\t所で', 'する\tて だけ\t泣いて いた事だけは', '始める\tで\tここで', '見る\tは を\t吾輩は ものを', '聞く\tで\tあとで', '捕える\tを\t我々を', '煮る\tて\t捕えて', '食う\tて\t煮て']


In [372]:
def _retrieve_src_joshi_surface(Chunk_ls, dst):
    src_joshi = []
    src_kou = []
    inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
    bases = np.asarray([
        morphs_base_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
    srcs = np.asarray([
        _Chunk.srcs for _Chunk in Chunk_ls])
    surfaces = np.asarray([
        morphs_face_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
    srcs = srcs[inds == dst][0]
    poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
    if len(srcs) == 0:
        return "", ""
    for src in srcs:
        src_pos = sum(
            poses[inds == src].tolist(),[])
        if "助詞" not in src_pos:
            continue
        src_bases = sum(
            bases[inds == src].tolist(), [])
        src_surfaces = sum(
            surfaces[inds == src].tolist(), [])
        src_joshi.append(src_surfaces[src_pos.index("助詞")])
        src_kou.append("".join(src_surfaces))
    return " ".join(src_joshi), " ".join(src_kou)

In [392]:
def nlp47():
    sentence_ls = nlp41()
    kaku_ls = []
    verb_kaku = ""
    src_joshi = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        bases = np.asarray([
            morphs_base_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        surfaces = np.asarray([
            morphs_face_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
        poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
        pos1s = np.asarray([
            morphs_pos1s(_Chunk.morphs) for _Chunk in Chunk_ls])
        for _Chunk in Chunk_ls:
            if len(_Chunk.srcs) == 0:
                continue
            else:
                src_kou = []
                src_joshi = []
                verb_kaku = ""
                has_sahen = False
                dst_poses = sum(
                    poses[inds == _Chunk.ind].tolist(),[])
                if "動詞" not in dst_poses:
                    continue
                dst_base = sum(
                        bases[inds == _Chunk.ind].tolist(), [])
                
                for src in _Chunk.srcs:
                    src_pos = sum(
                        poses[inds == src].tolist(),[])
                    if "助詞" not in src_pos:
                        continue
                    src_pos1s = sum(
                        pos1s[inds == src].tolist(), [])
                    if "サ変接続" not in src_pos1s:
                        continue
                    has_sahen = True
                    src_bases = sum(
                        bases[inds == src].tolist(), [])
                    src_surfaces = sum(
                        surfaces[inds == src].tolist(), [])
                    if src_surfaces[src_pos.index("助詞")] != "を":
                        has_sahen = False
                        continue
                    root_joshi, root_surface = _retrieve_src_joshi_surface(Chunk_ls, src)
                if has_sahen:
                    sahen = "".join(src_surfaces) + dst_base[dst_poses.index("動詞")]\
                        + "\t" + root_joshi + "\t" +root_surface
                    kaku_ls.append(sahen)
    return kaku_ls


if __name__ == "__main__":
    for i in nlp47()[:30]:
        print(i)

決心をする		
返報をする		
昼寝をする		
昼寝をする		
迫害を加える		
家族的生活をする		
話をする		
投書をする		
話をする	の	下のような
写生をする		
昼寝をする		
失笑するのを禁じる	て	見て
彩色を見る	の	今主人の
欠伸をする		
報道をする		
挨拶をする		
御馳走を食う		
問答をする	の	左の
雑談をする		
自慢話しを繰り返す	の	いつもの
自慢をする		
呼吸を飲み込む		
思案を定める		
御馳走をあるく	の	鼠以外の
御馳走を食う		
大分放蕩をする		
放蕩をする		
放蕩をする		
元来放蕩家をいう		
放蕩をする		


In [650]:
def nlp48():
    sentence_ls = nlp41()
    root_ls = []
    for Chunk_ls in sentence_ls:
        inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
        poses = np.asarray([
            morphs_poses(_Chunk.morphs) for _Chunk in Chunk_ls])
        root_ls.append([])
        root = ""
        for _Chunk in Chunk_ls:
            root = ""
            if _Chunk.dst == -1:
                continue
            if "名詞" not in poses[inds == _Chunk.ind]:
                continue
            root += morphs_face(_Chunk.morphs)
            ind = _Chunk.ind
            while True:
                dst_surfaces, dst_ind = _retrieve_dst_surface(Chunk_ls, ind)
                ind = dst_ind
                if ind == -1:
                    break
                root += "->" + dst_surfaces if root != "" else dst_surfaces
            root_ls[-1].append(root)
    return root_ls


if __name__ == "__main__":
    print(nlp48()[4])

['吾輩は->見た', 'ここで->始めて->人間という->ものを->見た', '人間という->ものを->見た', 'ものを->見た']


In [620]:
def _retrieve_dst_surface(Chunk_ls, src):
    inds = np.asarray([_Chunk.ind for _Chunk in Chunk_ls])
    bases = np.asarray([
        morphs_base_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
    surfaces = np.asarray([
        morphs_face_ls(_Chunk.morphs) for _Chunk in Chunk_ls])
    dsts = np.asarray([
        _Chunk.dst for _Chunk in Chunk_ls])
    dst = dsts[inds == src][0]
    if dst == -1:
        return "", -1
    try:
        if surfaces.dtype == "U3":
            surface = surfaces[inds == dst][0]
        else:
            surface = sum(surfaces[inds == dst].tolist(), [])
    except:
        print(surfaces[inds == dst])
        raise ValueError
    return "".join(surface), dst

In [788]:
def nlp49():
    root_ls = nlp48()
    path_ls = []
    for root in root_ls:
        path_ls.append([])
        path = ""
        root = [i.split("->") for i in root]
        combis = list(itertools.combinations(root, 2))
        for combi in combis:
            root_a = []
            root_b = []
            root_c = []
            for i in combi[0]:
                print(combi[0])
                if i not in combi[1]:
                    root_a.append(i)
                else:
                    if combi[1].index(i) != len(combi[1]) - 1:
                        root_a.append("Y")
                        break
                    root_b = combi[1][:combi[1].index(i)]
                    root_c = combi[1][combi[1].index(i):]
            if len(root_a) > 0:
                root_a[0] = noun_x(root_a[0], "X")
            if len(root_b) == 0:
                path_ls[-1].append("->".join(root_a))
            else:
                root_b[0] = noun_x(root_b[0], "Y")
                path_ls[-1].append("->".join(root_a) + "|" + "->".join(root_b) + "|" + root_c[0])
    return path_ls
if __name__ == "__main__":
    print(nlp49()[4])

['吾輩は', '見た']
['吾輩は', '見た']
['吾輩は', '見た']
['吾輩は', '見た']
['吾輩は', '見た']
['吾輩は', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['ここで', '始めて', '人間という', 'ものを', '見た']
['人間という', 'ものを', '見た']
['人間という', 'ものを', '見た']
['Xは|Yで->始めて->人間という->ものを|見た', 'Xは|Yという->ものを|見た', 'Xは|Yを|見た', 'Xで->始めて->Y', 'Xで->始めて->人間という->Y', 'Xという->Y']


In [642]:
hoge = ['吾輩は->見た', 'ここで->始めて->人間という->ものを->見た']
noun_ls = ["我輩は","ここで","人間という","ものを"]


In [643]:
import itertools

In [645]:
list(itertools.combinations(noun_ls, 2))

[('我輩は', 'ここで'),
 ('我輩は', '人間という'),
 ('我輩は', 'ものを'),
 ('ここで', '人間という'),
 ('ここで', 'ものを'),
 ('人間という', 'ものを')]

In [660]:
hoge = ['吾輩は->見た', 'ここで->始めて->人間という->ものを->見た', '人間という->ものを->見た', 'ものを->見た']
noun_ls = [i.split("->") for i in hoge]
hoge = list(itertools.combinations(noun_ls, 2))
hoge

[(['吾輩は', '見た'], ['ここで', '始めて', '人間という', 'ものを', '見た']),
 (['吾輩は', '見た'], ['人間という', 'ものを', '見た']),
 (['吾輩は', '見た'], ['ものを', '見た']),
 (['ここで', '始めて', '人間という', 'ものを', '見た'], ['人間という', 'ものを', '見た']),
 (['ここで', '始めて', '人間という', 'ものを', '見た'], ['ものを', '見た']),
 (['人間という', 'ものを', '見た'], ['ものを', '見た'])]

In [699]:
foo = hoge[0]
a = []
b = []
c = []
for k, i in enumerate(foo[0]):
    if i not in foo[1]:
        a.append(i)
    else:
        if foo[1].index(i) != len(foo[1]) - 1:
            a.append("Y")
            break
        b = foo[1][:foo[1].index(i)]
        c = foo[1][foo[1].index(i):]
print(a,b,c)

['吾輩は'] ['ここで', '始めて', '人間という', 'ものを'] ['見た']


In [786]:
def noun_x(text, replace):
    new_text = ""
    mecab = MeCab.Tagger()
    result = mecab.parse(text).split("\n")
    for res in result:
        if re.match(r"(\W|EOS)", res) is not None:
            continue
        if len(res.split()) == 0:
            continue
        if res.split()[1].split(",")[0] == "名詞":
            new_text += replace
        else:
            new_text += res.split()[0]
    return new_text

In [787]:
noun_x("吾輩は", "X")

'Xは'

In [781]:
mecab = MeCab.Tagger()
print(len(mecab.parse("吾輩は").split("\n")))

4


In [746]:
re.match(r"(\W|EOS)", "EOS")

<_sre.SRE_Match object; span=(0, 3), match='EOS'>